## Gestión de Cuencas Hidrográficas

En este presente trabajo generaremos un notebook que permita la automatizacion de distintos rater para mapas tematicos el cual nos permita visualizar y analizar información valiosa para la correcta gestión de las cuencas hidrográficas

Para usar este notebook se debe ingresar los pirmeros parametros (DEM y shape) arrastrandolos hacia el notebook (si usa arcgis pro) después ejecutrar celda por celda de cada uno de los mapas , todos los resultados se alojan en el database predefinido del proyecto

### Primeros pasos 

In [1]:
import arcpy
import os
from arcpy.sa import * # herramientas de análisis espacial

# 1. Configuración General del Entorno
# Permite sobreescribir archivos de salida si se re-ejecuta el código
arcpy.env.overwriteOutput = True

# Define el espacio de trabajo predeterminado como la Geodatabase del proyecto actual
aprx = arcpy.mp.ArcGISProject("CURRENT")
gdb_trabajo = aprx.defaultGeodatabase
arcpy.env.workspace = gdb_trabajo

# 2. Gestión de Licencias
# Verifica y activa la extensión necesaria para cálculos de superficie e hidrología
if arcpy.CheckExtension("Spatial") == "Available":
    arcpy.CheckOutExtension("Spatial")
    print(f"Entorno configurado correctamente.\nWorkspace: {gdb_trabajo}")
else:
    raise RuntimeError("Error Crítico: La extensión Spatial Analyst no está disponible.")

Entorno configurado correctamente.
Workspace: C:\Users\Cesar_2borgbt\Documents\SIG\Portafolio_Ingenieria\Gestion_cuencas_hidrograficas\Manejo_de_cuencas_Hidrograficas\Manejo_de_cuencas_Hidrograficas.gdb


### Ingresamos nuestro modelo de elevacion digital (DEM) y nuestro shape de la cuneca delimitada

In [2]:

# 1. Modelo de Elevación Digital (Raster)
# Debe ser el DEM ya corregido o el mosaico final que generamos anteriormente
input_dem =   "Mosaico_DEM_aguaytia_Clip"

# 2. Límite de la Cuenca (Vector/Polígono)
# Shapefile o Feature Class que define el área de estudio
input_cuenca =  "cuenca Aguaytia"

# --- VALIDACIÓN DE DATOS ---
inputs_validos = True
recursos = {"DEM": input_dem, "Cuenca": input_cuenca}

for tipo, capa in recursos.items():
    if not arcpy.Exists(capa):
        print(f"Error: No se encuentra la capa de entrada '{tipo}': {capa}")
        inputs_validos = False

if inputs_validos:
    # Describe las propiedades espaciales para asegurar consistencia
    desc_dem = arcpy.Describe(input_dem)
    cell_size = desc_dem.meanCellWidth
    print(f"Datos validados exitosamente.")
    print(f"Resolución espacial del DEM: {cell_size} metros.")
    print(f"Sistema de Coordenadas: {desc_dem.spatialReference.name}")
else:
    raise FileNotFoundError("Deteniendo ejecución: Verifique los nombres de las capas de entrada.")

Datos validados exitosamente.
Resolución espacial del DEM: 0.00027777777777777794 metros.
Sistema de Coordenadas: GCS_WGS_1984


### 1.- Mapa de pendientes

In [3]:
def generar_mapa_pendientes(dem_raster, cuenca_mask):
    """
    Calcula la pendiente en porcentaje y la reclasifica según rangos estandarizados.
    Retorna: Ruta del raster de pendiente y ruta del raster reclasificado.
    """
    # 1. Definición de rutas de salida
    out_slope_name = "Pendiente_Porcentaje"
    out_reclass_name = "Pendiente_Reclasificada"
    
    try:
        print("Calculando pendiente (Slope) en porcentaje...")
        # Cálculo de pendiente
        # Output measurement: PERCENT_RISE (para coincidir con la tabla de rangos %)
        slope_raster = Slope(dem_raster, "PERCENT_RISE")
        
        # Guardar raster de pendiente cruda
        slope_raster.save(out_slope_name)
        
        print("Reclasificando pendiente según tabla de rangos...")
        # 2. Configuración de Rangos de Reclasificación (RemapRange)
        # Formato: [min, max, valor_nuevo]
        # Rangos basados en tabla: 0-4, 4-8, 8-15, 15-25, 25-50, 50-75, >75
        remap = RemapRange([
            [0, 4, 1],
            [4, 8, 2],
            [8, 15, 3],
            [15, 25, 4],
            [25, 50, 5],
            [50, 75, 6],
            [75, 10000, 7]  # Límite superior arbitrario alto para cubrir todo >75
        ])
        
        # Ejecución de Reclassify
        reclass_raster = Reclassify(slope_raster, "VALUE", remap)
        
        # 3. Recorte final con la máscara de la cuenca (para asegurar bordes limpios)
        # Si el DEM de entrada ya está cortado, este paso asegura que la salida sea exacta.
        final_reclass = ExtractByMask(reclass_raster, cuenca_mask)
        final_reclass.save(out_reclass_name)
        
        return out_slope_name, out_reclass_name

    except Exception as e:
        return f"Error en el proceso de pendientes: {str(e)}", None

# --- EJECUCIÓN DEL MÓDULO ---
if inputs_validos:
    # Usamos las variables globales definidas en la Celda 2
    r_pendiente, r_reclasificado = generar_mapa_pendientes(input_dem, input_cuenca)
    
    if r_reclasificado:
        print(f"Proceso finalizado exitosamente.")
        print(f"1. Raster Pendiente Continua: {r_pendiente}")
        print(f"2. Raster Reclasificado: {r_reclasificado}")
        
        # Diccionario de referencia para simbología posterior (Metadatos)
        leyenda_pendiente = {
            1: "Plana a Ligeramente inclinada (0-4%)",
            2: "Moderadamente inclinada (4-8%)",
            3: "Fuertemente inclinada (8-15%)",
            4: "Moderadamente empinada (15-25%)",
            5: "Empinada (25-50%)",
            6: "Muy Empinada (50-75%)",
            7: "Extremadamente empinada (>75%)"
        }
        print("Diccionario de clases generado para referencia.")
    else:
        print(r_pendiente) # Imprime el error si ocurrió

#agregar diccionario a tabla de atributos

def agregar_descripciones_tabla(raster_entrada, diccionario_clases):
    """
    Agrega un campo de texto a la tabla del raster y lo puebla 
    basado en un diccionario de referencia (Value -> Descripción).
    """
    campo_desc = "Descripcion_Clase"
    
    try:
        # 1. Agregar el campo de texto si no existe
        # FIELD_TYPE: TEXT, LENGTH: 100 caracteres
        arcpy.management.AddField(raster_entrada, campo_desc, "TEXT", field_length=100)
        
        # 2. Actualizar las filas usando un Cursor (UpdateCursor)
        # Se iteran las filas buscando coincidencias entre 'Value' y el diccionario
        with arcpy.da.UpdateCursor(raster_entrada, ["Value", campo_desc]) as cursor:
            for row in cursor:
                valor_pixel = row[0] # Columna Value
                if valor_pixel in diccionario_clases:
                    row[1] = diccionario_clases[valor_pixel] # Asigna la descripción
                    cursor.updateRow(row)
                    
        print(f"Tabla actualizada correctamente. Campo '{campo_desc}' generado.")
        
    except Exception as e:
        print(f"Error actualizando tabla: {str(e)}")

# --- EJECUCIÓN ---
if inputs_validos and 'r_reclasificado' in locals():
    # Usamos el raster generado en la celda anterior y el diccionario de leyenda
    # Asegúrese de que la capa "Pendiente_Reclasificada" no esté abierta/bloqueada en la vista
    agregar_descripciones_tabla(r_reclasificado, leyenda_pendiente)

Calculando pendiente (Slope) en porcentaje...
Reclasificando pendiente según tabla de rangos...
Proceso finalizado exitosamente.
1. Raster Pendiente Continua: Pendiente_Porcentaje
2. Raster Reclasificado: Pendiente_Reclasificada
Diccionario de clases generado para referencia.


### 2.- Mapa de Capacidad de uso Mayor (CUM)

In [5]:
def generar_capacidad_uso_mayor(raster_pendiente_pct):
    """
    Genera el mapa de Capacidad de Uso Mayor (CUM) basado en rangos de pendiente (Ladera Corta).
    Agrega campos 'Simbolo' y 'Categoria' a la tabla de atributos.
    """
    out_cum_name = "Capacidad_Uso_Mayor"
    
    try:
        print("Clasificando Capacidad de Uso Mayor...")
        
        # 1. Configuración de Rangos según metodología (Imagen suministrada)
        # 0-4% -> A, 4-8% -> C, 8-25% -> P, 25-50% -> F, >50% -> X
        remap_cum = RemapRange([
            [0, 4, 1],
            [4, 8, 2],
            [8, 25, 3],
            [25, 50, 4],
            [50, 10000, 5]
        ])
        
        # Ejecución de Reclassify
        cum_raster = Reclassify(raster_pendiente_pct, "VALUE", remap_cum)
        cum_raster.save(out_cum_name)
        
        # 2. Enriquecimiento de Atributos (Diccionario integrado)
        # Estructura: Value: (Simbolo, Descripción Técnica)
        metadata_cum = {
            1: ("A", "Tierras Aptas para Cultivo en Limpio"),
            2: ("C", "Tierras Aptas para Cultivos Permanentes"),
            3: ("P", "Tierras Aptas para Pastos"),
            4: ("F", "Tierras Aptas para Producción Forestal"),
            5: ("X", "Tierras de Protección")
        }
        
        # Agregar campos
        arcpy.management.AddField(out_cum_name, "Simbolo", "TEXT", field_length=5)
        arcpy.management.AddField(out_cum_name, "Categoria", "TEXT", field_length=100)
        
        # Poblar tabla
        with arcpy.da.UpdateCursor(out_cum_name, ["Value", "Simbolo", "Categoria"]) as cursor:
            for row in cursor:
                val = row[0]
                if val in metadata_cum:
                    row[1] = metadata_cum[val][0] # Asigna Símbolo (A, C, P...)
                    row[2] = metadata_cum[val][1] # Asigna Descripción
                    cursor.updateRow(row)
                    
        return out_cum_name

    except Exception as e:
        return f"Error en proceso CUM: {str(e)}"

# --- EJECUCIÓN ---
if inputs_validos:
    # Se requiere el raster de 'Pendiente_Porcentaje' generado en la Celda 3
    # Si la variable 'r_pendiente' se perdió de la memoria, use el string "Pendiente_Porcentaje"
    input_pendiente_pct = "Pendiente_Porcentaje"
    
    if arcpy.Exists(input_pendiente_pct):
        r_cum = generar_capacidad_uso_mayor(input_pendiente_pct)
        print(f"Proceso finalizado.\nCapa generada: {r_cum}")
        print("Verifique la tabla de atributos para visualizar los campos 'Simbolo' y 'Categoria'.")
    else:
        print("Error: No se encuentra el raster de Pendiente Porcentaje necesario para este cálculo.")

Clasificando Capacidad de Uso Mayor...
Proceso finalizado.
Capa generada: Capacidad_Uso_Mayor
Verifique la tabla de atributos para visualizar los campos 'Simbolo' y 'Categoria'.


### 3.- Mapa de Altitudes

In [6]:
def generar_pisos_altitudinales(dem_raster, cuenca_mask):
    """
    Clasifica el DEM en Pisos Altitudinales según la tesis de las 
    8 Regiones Naturales del Perú (Dr. Javier Pulgar Vidal).
    """
    out_pisos_name = "Pisos_Altitudinales"
    
    try:
        print("Clasificando Pisos Altitudinales (Zonificación Hipsométrica)...")
        
        # 1. Configuración de Rangos de Altitud (Metros sobre el nivel del mar)
        # Basado en la tabla de referencia suministrada.
        # Nota: Se utiliza el límite superior técnico de los Andes (~6768m) para la última clase.
        remap_altitud = RemapRange([
            [0, 500, 1],       # Costa o Chala
            [500, 2300, 2],    # Yunga
            [2300, 3500, 3],   # Quechua
            [3500, 4000, 4],   # Suni
            [4000, 4800, 5],   # Puna
            [4800, 6768, 6]    # Janca o Cordillera (Jalca en referencia)
        ])
        
        # Ejecución de Reclassify
        pisos_raster = Reclassify(dem_raster, "VALUE", remap_altitud)
        
        # Recorte final por máscara de cuenca
        final_pisos = ExtractByMask(pisos_raster, cuenca_mask)
        final_pisos.save(out_pisos_name)
        
        # 2. Enriquecimiento de Atributos
        metadata_pisos = {
            1: ("Costa o Chala", "0 - 500 m.s.n.m."),
            2: ("Yunga", "500 - 2300 m.s.n.m."),
            3: ("Quechua", "2300 - 3500 m.s.n.m."),
            4: ("Suni", "3500 - 4000 m.s.n.m."),
            5: ("Puna", "4000 - 4800 m.s.n.m."),
            6: ("Janca o Cordillera", "> 4800 m.s.n.m.")
        }
        
        # Agregar campos descriptivos
        arcpy.management.AddField(out_pisos_name, "Region_Natural", "TEXT", field_length=50)
        arcpy.management.AddField(out_pisos_name, "Rango_Altitud", "TEXT", field_length=50)
        
        # Poblar tabla
        with arcpy.da.UpdateCursor(out_pisos_name, ["Value", "Region_Natural", "Rango_Altitud"]) as cursor:
            for row in cursor:
                val = row[0]
                if val in metadata_pisos:
                    row[1] = metadata_pisos[val][0]
                    row[2] = metadata_pisos[val][1]
                    cursor.updateRow(row)
                    
        return out_pisos_name

    except Exception as e:
        return f"Error en Pisos Altitudinales: {str(e)}"

# --- EJECUCIÓN ---
if inputs_validos:
    # Utiliza el DEM original (elevación pura), NO el de pendientes
    r_pisos = generar_pisos_altitudinales(input_dem, input_cuenca)
    
    if not r_pisos.startswith("Error"):
        print(f"Proceso finalizado.\nCapa generada: {r_pisos}")
        print("Tabla de atributos actualizada con Región Natural y Rango.")
    else:
        print(r_pisos)

Clasificando Pisos Altitudinales (Zonificación Hipsométrica)...
Proceso finalizado.
Capa generada: Pisos_Altitudinales
Tabla de atributos actualizada con Región Natural y Rango.


### 4.- Mapa de Orientacion (Aspect Map)

In [7]:
def generar_mapa_orientacion(dem_raster, cuenca_mask):
    """
    Calcula la exposición (Aspect) y la reclasifica en 8 direcciones cardinales + Plano.
    """
    out_aspect_name = "Orientacion_Laderas"
    
    try:
        print("Calculando Orientación (Aspect)...")
        
        # 1. Cálculo de Aspecto (Grados 0-360)
        aspect_raw = Aspect(dem_raster)
        
        # 2. Reclasificación en 8 Direcciones Cardinales
        # El Norte tiene dos rangos: 0-22.5 y 337.5-360. Ambos se asignan al valor 1.
        # Código: -1=Plano, 1=N, 2=NE, 3=E, 4=SE, 5=S, 6=SO, 7=O, 8=NO
        remap_aspect = RemapRange([
            [-1, 0, 0],         # Plano
            [0, 22.5, 1],       # Norte (Parte 1)
            [22.5, 67.5, 2],    # Noreste
            [67.5, 112.5, 3],   # Este
            [112.5, 157.5, 4],  # Sureste
            [157.5, 202.5, 5],  # Sur
            [202.5, 247.5, 6],  # Suroeste
            [247.5, 292.5, 7],  # Oeste
            [292.5, 337.5, 8],  # Noroeste
            [337.5, 360, 1]     # Norte (Parte 2)
        ])
        
        aspect_reclass = Reclassify(aspect_raw, "VALUE", remap_aspect)
        
        # Recorte final
        final_aspect = ExtractByMask(aspect_reclass, cuenca_mask)
        final_aspect.save(out_aspect_name)
        
        # 3. Enriquecimiento de Atributos
        metadata_orientacion = {
            0: "Plano (Sin pendiente)",
            1: "Norte (N)",
            2: "Noreste (NE)",
            3: "Este (E)",
            4: "Sureste (SE)",
            5: "Sur (S)",
            6: "Suroeste (SO)",
            7: "Oeste (O)",
            8: "Noroeste (NO)"
        }
        
        # Agregar campo descriptivo
        arcpy.management.AddField(out_aspect_name, "Direccion", "TEXT", field_length=30)
        
        with arcpy.da.UpdateCursor(out_aspect_name, ["Value", "Direccion"]) as cursor:
            for row in cursor:
                val = row[0]
                if val in metadata_orientacion:
                    row[1] = metadata_orientacion[val]
                    cursor.updateRow(row)
                    
        return out_aspect_name

    except Exception as e:
        return f"Error en Aspect: {str(e)}"

# --- EJECUCIÓN ---
if inputs_validos:
    # Usa el DEM original de elevación
    r_orientacion = generar_mapa_orientacion(input_dem, input_cuenca)
    
    if not r_orientacion.startswith("Error"):
        print(f"Proceso finalizado.\nCapa generada: {r_orientacion}")
        print("Tabla de atributos actualizada con direcciones cardinales.")
    else:
        print(r_orientacion)

Calculando Orientación (Aspect)...
Proceso finalizado.
Capa generada: Orientacion_Laderas
Tabla de atributos actualizada con direcciones cardinales.


### 5.- Mapa de Sombreado (Hillshade)

In [8]:
def generar_hillshade(dem_raster, cuenca_mask, azimut=315, altitud=45):
    """
    Genera el modelo de sombreado (Hillshade) para efecto de relieve.
    Parámetros estándar: Azimut 315, Altitud 45.
    """
    out_hs_name = f"Mapa_Sombreado_Az{azimut}_Alt{altitud}"
    
    try:
        print(f"Calculando Hillshade (Azimut: {azimut}, Altitud: {altitud})...")
        
        # 1. Ejecución de la herramienta Hillshade
        # model_shadows="NO_SHADOWS" (Por defecto, más rápido y limpio para cartografía base)
        hs_raster = Hillshade(dem_raster, azimut, altitud, "NO_SHADOWS", 1)
        
        # 2. Recorte final por máscara de cuenca
        final_hs = ExtractByMask(hs_raster, cuenca_mask)
        final_hs.save(out_hs_name)
        
        return out_hs_name

    except Exception as e:
        return f"Error en Hillshade: {str(e)}"

# --- EJECUCIÓN ---
if inputs_validos:
    # Generación con parámetros estándar
    r_sombreado = generar_hillshade(input_dem, input_cuenca)
    
    if not r_sombreado.startswith("Error"):
        print(f"Proceso finalizado.\nCapa generada: {r_sombreado}")
        print("Nota: Utilice esta capa con 30-50% de transparencia sobre otros mapas para dar efecto 3D.")
    else:
        print(r_sombreado)

Calculando Hillshade (Azimut: 315, Altitud: 45)...
Proceso finalizado.
Capa generada: Mapa_Sombreado_Az315_Alt45
Nota: Utilice esta capa con 30-50% de transparencia sobre otros mapas para dar efecto 3D.


### 6.- Mapa de Densidad de Drenaje.

In [11]:
def generar_densidad_drenaje(dem_raster, cuenca_mask, umbral_acumulacion=1000):
    """
    Genera el Mapa de Densidad de Drenaje (km de río / km2).
    Usa la GDB activa para temporales.
    """
    # CORRECCIÓN: Definimos gdb_path leyendo el workspace actual
    gdb_path = arcpy.env.workspace 
    out_density_name = "Densidad_Drenaje"
    temp_streams = f"{gdb_path}\\temp_streams_densidad" 
    
    try:
        print(f"1. Generando red de drenaje temporal en: {temp_streams}...")
        
        # Procesamiento hidrológico
        fill = Fill(dem_raster)
        fdir = FlowDirection(fill, "NORMAL", None, "D8")
        facc = FlowAccumulation(fdir, None, "FLOAT", "D8")
        
        # Definición de cauces
        streams_ras = Con(facc > umbral_acumulacion, 1)
        
        # Conversión a Vector (Polyline)
        # Se guarda físicamente para asegurar que LineDensity lo encuentre
        StreamToFeature(streams_ras, fdir, temp_streams, "SIMPLIFY")
        
        # Verificación de seguridad: ¿Se crearon líneas?
        count_result = arcpy.management.GetCount(temp_streams)
        if int(count_result[0]) == 0:
            return "Error: No se generaron líneas. El umbral es muy alto o el DEM es plano."

        print("2. Calculando Densidad de Líneas (km/km2)...")
        # Line Density
        densidad_raw = LineDensity(temp_streams, "NONE", dem_raster)
        
        # Recorte final
        final_density = ExtractByMask(densidad_raw, cuenca_mask)
        final_density.save(out_density_name)
        
        # Limpieza
        arcpy.management.Delete(temp_streams)
        
        return out_density_name

    except Exception as e:
        # Intento de limpieza si falla
        if arcpy.Exists(temp_streams):
            arcpy.management.Delete(temp_streams)
        return f"Error en Densidad de Drenaje: {str(e)}"

# --- EJECUCIÓN ---
if inputs_validos:
    r_densidad = generar_densidad_drenaje(input_dem, input_cuenca, umbral_acumulacion=1000)
    
    if not r_densidad.startswith("Error"):
        print(f"Proceso finalizado.\nCapa generada: {r_densidad}")
    else:
        print(r_densidad)

1. Generando red de drenaje temporal en: C:\Users\Cesar_2borgbt\Documents\SIG\Portafolio_Ingenieria\Gestion_cuencas_hidrograficas\Manejo_de_cuencas_Hidrograficas\Manejo_de_cuencas_Hidrograficas.gdb\temp_streams_densidad...
2. Calculando Densidad de Líneas (km/km2)...
Proceso finalizado.
Capa generada: Densidad_Drenaje


In [13]:
import math

def calcular_morfometria(cuenca_feature):
    """
    Calcula morfometría proyectando al vuelo a UTM Zona 18S (Perú Central).
    Soluciona el error de área 0.00 en capas geográficas.
    """
    # Código EPSG para WGS 1984 UTM Zone 18S (Rupa-Rupa/Tingo María)
    sr_utm_peru = arcpy.SpatialReference(32718)
    
    try:
        print("--- REPORTE MORFOMÉTRICO (Proyectado a UTM 18S) ---")
        
        # Usamos el cursor con 'spatial_reference' para medir en METROS reales
        # SHAPE@ proyecta la geometría en memoria antes de devolver el valor
        with arcpy.da.SearchCursor(cuenca_feature, ["SHAPE@"], spatial_reference=sr_utm_peru) as cursor:
            for row in cursor:
                geom = row[0]
                area_m2 = geom.area
                perimetro_m = geom.length
                
        # 2. Conversión de Unidades
        area_km2 = area_m2 / 1_000_000
        perimetro_km = perimetro_m / 1_000
        
        print(f"1. Área de Drenaje (A): \t{area_km2:.2f} km²")
        print(f"2. Perímetro Total (P): \t{perimetro_km:.2f} km")
        
        # 3. Cálculo del Índice de Compacidad (Gravelius) - Kc
        # Fórmula: Kc = 0.282 * (P / Raíz(A))
        if area_km2 > 0:
            kc = 0.282 * (perimetro_km / math.sqrt(area_km2))
            print(f"3. Índice de Gravelius (Kc): \t{kc:.3f}")
            
            # 4. Interpretación
            if kc < 1.12:
                diag = "Cuenca REDONDA (Respuesta Rápida / Pico Alto)"
            elif 1.12 <= kc < 1.50:
                diag = "Cuenca OVAL - OBLONGA"
            else:
                diag = "Cuenca ALARGADA (Respuesta Lenta / Pico Bajo)"
            print(f"   -> Diagnóstico: {diag}")
        else:
            print("Error: El área sigue siendo 0. Verifique la geometría.")

    except Exception as e:
        print(f"Error en morfometría: {str(e)}")

# --- EJECUCIÓN ---
if inputs_validos:
    calcular_morfometria(input_cuenca)

--- REPORTE MORFOMÉTRICO (Proyectado a UTM 18S) ---
1. Área de Drenaje (A): 	11352.87 km²
2. Perímetro Total (P): 	689.93 km
3. Índice de Gravelius (Kc): 	1.826
   -> Diagnóstico: Cuenca ALARGADA (Respuesta Lenta / Pico Bajo)
